In [1]:
import requests
import pandas as pd
import time

API_KEY = '9MBUP7GPCWD3VV9S'
symbol = "AAPL"

# Fetch Income Statement (Revenue, Net Income, EPS) - Quarterly
url_income = f"https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&apikey={API_KEY}"
response = requests.get(url_income)
income_data = response.json()
time.sleep(12)  # To avoid API limit

# Fetch Balance Sheet (Debt, Equity, Assets) - Quarterly
url_balance = f"https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&apikey={API_KEY}"
response = requests.get(url_balance)
balance_data = response.json()
time.sleep(12)  # API rate limit

# Fetch Cash Flow Statement (Operating Cash Flow, CapEx) - Quarterly
url_cashflow = f"https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&apikey={API_KEY}"
response = requests.get(url_cashflow)
cashflow_data = response.json()


In [2]:
# Extract quarterly income statement (last 40 quarters if available)
income_quarters = income_data["quarterlyReports"][:40]  # 10 years

# Store in DataFrame
income_df = pd.DataFrame({
    "Date": [entry["fiscalDateEnding"] for entry in income_quarters],
    "Revenue": [float(entry["totalRevenue"]) if entry["totalRevenue"] else None for entry in income_quarters],
    "Net Income": [float(entry["netIncome"]) if entry["netIncome"] else None for entry in income_quarters],
    "EPS": [float(entry.get("dilutedEPS", 0)) for entry in income_quarters]  # Handle missing EPS
})


In [3]:
# Extract quarterly balance sheet (last 40 quarters)
balance_quarters = balance_data["quarterlyReports"][:40]

# Store in DataFrame
balance_df = pd.DataFrame({
    "Date": [entry["fiscalDateEnding"] for entry in balance_quarters],
    "Total Assets": [float(entry["totalAssets"]) if entry["totalAssets"] else None for entry in balance_quarters],
    "Total Liabilities": [float(entry["totalLiabilities"]) if entry["totalLiabilities"] else None for entry in balance_quarters],
    "Total Equity": [float(entry["totalShareholderEquity"]) if entry["totalShareholderEquity"] else None for entry in balance_quarters],
     "Total Debt": [float(entry.get("totalDebt", 0)) for entry in balance_quarters]
})


In [4]:
# Extract quarterly cash flow statement (last 40 quarters)
cashflow_quarters = cashflow_data["quarterlyReports"][:40]

# Store in DataFrame
cashflow_df = pd.DataFrame({
    "Date": [entry["fiscalDateEnding"] for entry in cashflow_quarters],
    "Operating Cash Flow": [float(entry["operatingCashflow"]) if entry["operatingCashflow"] else None for entry in cashflow_quarters],
    "Capital Expenditures": [float(entry["capitalExpenditures"]) if entry["capitalExpenditures"] else None for entry in cashflow_quarters]
})

# Calculate Free Cash Flow (FCF)
cashflow_df["Free Cash Flow"] = cashflow_df["Operating Cash Flow"] + cashflow_df["Capital Expenditures"]


In [5]:
# Merge all DataFrames on Date
financial_df = income_df.merge(balance_df, on="Date").merge(cashflow_df, on="Date")

# Calculate Financial Ratios
financial_df["Profit Margin (%)"] = (financial_df["Net Income"] / financial_df["Revenue"]) * 100
financial_df["ROE (%)"] = (financial_df["Net Income"] / financial_df["Total Equity"]) * 100
financial_df["ROA (%)"] = (financial_df["Net Income"] / financial_df["Total Assets"]) * 100
financial_df["Debt/Equity"] = financial_df["Total Debt"] / financial_df["Total Equity"]
financial_df["FCF Margin (%)"] = (financial_df["Free Cash Flow"] / financial_df["Revenue"]) * 100

# Display the DataFrame
print(financial_df)  # Print DataFrame in console
financial_df.to_csv("/tmp/financial_ratios_10_years_quarterly.csv", index=False)
print("File saved to /tmp/")


          Date       Revenue    Net Income  EPS  Total Assets  \
0   2024-12-31  1.243000e+11  3.633000e+10  0.0  3.440850e+11   
1   2024-09-30  9.493000e+10  1.473600e+10  0.0  3.649800e+11   
2   2024-06-30  8.577700e+10  2.144800e+10  0.0  3.316120e+11   
3   2024-03-31  9.075300e+10  2.363600e+10  0.0  3.374110e+11   
4   2023-12-31  1.195750e+11  3.391600e+10  0.0  3.535140e+11   
5   2023-09-30  8.849600e+10  2.295600e+10  0.0  3.525830e+11   
6   2023-06-30  8.079900e+10  1.988100e+10  0.0  3.350380e+11   
7   2023-03-31  9.483600e+10  2.416000e+10  0.0  3.321600e+11   
8   2023-03-31  9.483600e+10  2.416000e+10  0.0  3.321600e+11   
9   2022-12-31  1.161510e+11  2.999800e+10  0.0  3.467470e+11   
10  2022-09-30  8.931900e+10  2.072100e+10  0.0  3.527550e+11   
11  2022-06-30  8.224000e+10  1.944200e+10  0.0  3.363090e+11   
12  2022-03-31  9.658700e+10  2.501000e+10  0.0  3.506620e+11   
13  2021-12-31  1.232510e+11  3.463000e+10  0.0  3.811910e+11   
14  2021-09-30  8.268800e

In [16]:
import pandas as pd

# Define file paths
earnings_file_path = "apple_earnings.csv"
price_file_path = "AAPL_daily_data.csv"
financial_file_path = "financial_ratios_10_years_quarterly.csv"

# Load earnings announcement data
earnings_df = pd.read_csv(earnings_file_path)

# Rename columns for consistency
earnings_df.rename(columns={"Fiscal Quarter End": "Fiscal End Date", "Announcement Date": "Announcement Date"}, inplace=True)

# Convert date columns to datetime format
earnings_df["Fiscal End Date"] = pd.to_datetime(earnings_df["Fiscal End Date"])
earnings_df["Announcement Date"] = pd.to_datetime(earnings_df["Announcement Date"])

# Load financial ratios dataset
financial_df = pd.read_csv(financial_file_path)

# Convert dates in financial data
financial_df["Fiscal End Date"] = pd.to_datetime(financial_df["Date"])

# Merge financial ratios with earnings announcement dates using the fiscal period
financial_with_earnings = pd.merge(financial_df, earnings_df, on="Fiscal End Date", how="left")

# Ensure 'Announcement Date' is sorted before merging
financial_with_earnings.sort_values("Announcement Date", inplace=True)

# Load stock price data
price_df = pd.read_csv(price_file_path)

# Rename and clean price_df
if "Unnamed: 0" in price_df.columns:
    price_df.rename(columns={"Unnamed: 0": "Date"}, inplace=True)

# Convert Date column to datetime format
price_df["Date"] = pd.to_datetime(price_df["Date"])

# Ensure sorting before merging
price_df.sort_values("Date", inplace=True)

# Merge stock prices with financial ratios using fiscal quarter end date
merged_df = pd.merge_asof(
    price_df, 
    financial_with_earnings, 
    left_on="Date", 
    right_on="Announcement Date", 
    direction="backward"
)

# Drop unnecessary columns
merged_df.drop(columns=["Announcement Date"], inplace=True)

# Save final merged dataset
merged_file_path = "AAPL_merged_data.csv"
merged_df.to_csv(merged_file_path, index=False)

# Display the merged dataset to the user

# Return the file path for downloading
print(f"Merged dataset saved to: {merged_file_path}")


Merged dataset saved to: AAPL_merged_data.csv
